In [ ]:
import os
import shutil
from tqdm import tqdm

OLD_ROOT = "/data/TimeSeriesResearch/datasets/Satya"
NEW_ROOT = "/data/TimeSeriesResearch/datasets/Satya_New"

def restructure_dataset(old_root=OLD_ROOT, new_root=NEW_ROOT):
    os.makedirs(new_root, exist_ok=True)

    for folder in tqdm(os.listdir(old_root), desc="Processing Folders"):
        folder_path = os.path.join(old_root, folder)
        
        if not os.path.isdir(folder_path):
            continue

        for file_name in os.listdir(folder_path):
            if not file_name.lower().endswith(".csv"):
                continue
            
            parts = file_name.split('_')
            if len(parts) >= 3:
                label = '_'.join(parts[2:]).replace('.csv', '')
                index = parts[1]
            else:
                label = "Unknown"
                index = parts[1] if len(parts) > 1 else "0"

            label_folder = os.path.join(new_root, folder, label)
            os.makedirs(label_folder, exist_ok=True)

            new_file_name = f"file_{index}.csv"
            src_path = os.path.join(folder_path, file_name)
            dst_path = os.path.join(label_folder, new_file_name)

            try:
                shutil.copy(src_path, dst_path)
                print(f"Moved: {src_path} -> {dst_path}")
            except Exception as e:
                print(f"Error moving {src_path} to {dst_path}: {e}")

    print("Restructuring complete.")

restructure_dataset()


In [ ]:
### File movement
import os
import shutil
import pandas as pd
from tqdm import tqdm

# Paths
source_dir = "/data/TimeSeriesResearch/datasets/kaggle/processed/processed_datasets_labelled_separated/SKAB - Skoltech Anomaly Benchmark"
dest_dir = "/data/TimeSeriesResearch/datasets/Satya_New/SKAB - Skoltech Anomaly Benchmark"

# Ensure the destination directory exists
os.makedirs(dest_dir, exist_ok=True)

# Dictionary to keep track of the file counts for each label
label_count = {}

# Process each file in the source directory
for file_name in tqdm(os.listdir(source_dir), desc="Processing files"):
    # Check if it's a CSV file
    if not file_name.endswith(".csv"):
        continue

    # Split the file name to extract the label
    parts = file_name.split('_')
    label = parts[-1].replace('.csv', '').strip()
    
    # Create the label directory if it doesn't exist
    label_dir = os.path.join(dest_dir, label)
    os.makedirs(label_dir, exist_ok=True)

    # Keep count of files within each label
    if label not in label_count:
        label_count[label] = 0

    # Construct the new file name based on the order
    new_file_name = f"file_{label_count[label]}.csv"
    label_count[label] += 1

    # Full paths for reading, processing, and moving
    src_path = os.path.join(source_dir, file_name)
    dest_path = os.path.join(label_dir, new_file_name)

    # Load the CSV and drop the label column
    try:
        df = pd.read_csv(src_path)
        if "Label" in df.columns:
            df = df.drop(columns=["Label"], errors='ignore')

        # Save the modified file in the new location
        df.to_csv(dest_path, index=False)

    except Exception as e:
        print(f"Error processing {file_name}: {e}")

print("Files processed and moved successfully!")


In [45]:
from collections import defaultdict
from ftse.data.Dataset import UnwindowedDataset
from tqdm.auto import tqdm
import os
import h5py
import numpy as np
import pandas as pd
import torch

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"


ROOT_DIRS = [
    '/data/TimeSeriesResearch/datasets/Satya_New'
]

H5_FILE = '/data/TimeSeriesResearch/datasets/satya_data2.h5'

def create_h5py_dataset_from_root_dirs(root_dirs=ROOT_DIRS, h5_file=H5_FILE):
    with h5py.File(h5_file, "w") as h5f:
        for root_dir in root_dirs:
            root_prefix = os.path.basename(os.path.normpath(root_dir))

            for dataset_folder in tqdm(os.listdir(root_dir), desc=f"Datasets in {root_dir}"):
                dataset_path = os.path.join(root_dir, dataset_folder)
                if not os.path.isdir(dataset_path):
                    print("skipping", dataset_path)
                    print("\n"*50)
                    continue
                    
                #For labels
                for subfolder in os.listdir(dataset_path):
                    subfolder_path = os.path.join(dataset_path, subfolder)
                    if not os.path.isdir(subfolder_path):
                        continue

                    for csv_file in os.listdir(subfolder_path):
                        if not csv_file.lower().endswith(".csv"):
                            continue
                            
                        group_path = f"{root_prefix}/{dataset_folder}/{subfolder}"
                        group = h5f.require_group(group_path)
                        #print("group",group)

                        csv_path = os.path.join(subfolder_path, csv_file)
                        df = pd.read_csv(csv_path)

                        drop_columns = ['Unnamed: 0', 'time_sec', 'Time', 'Label']
                        df = df.drop(columns=[col for col in drop_columns if col in df.columns], errors='ignore')

                        column_names = df.columns.tolist()
                        data_array = df.to_numpy()
                        dataset_name = f"file_{len(group)}"
                        #print(group_path, "--", dataset_name)
                        dset = group.create_dataset(dataset_name, data=data_array)
                        dset.attrs["descriptions"] = list(column_names)
                        #print(f"Created dataset at: {group_path}/{dataset_name}")
    print("Flat HDF5 file created successfully:", h5_file)

Processing /data/TimeSeriesResearch/datasets/Satya_New:   0%|                                                                                                                                                                                                             | 0/18 [00:00<?, ?it/s]

Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_0
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_1
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_2
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_3
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_4
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_5
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_6
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_7
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_8
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_9
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_10
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/valve1/file_11
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmar

Processing /data/TimeSeriesResearch/datasets/Satya_New:   6%|██████████▉                                                                                                                                                                                          | 1/18 [00:00<00:04,  3.48it/s]

Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_38
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_39
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_40
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_41
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_42
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_43
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_44
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_45
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_46
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_47
Created dataset at: Satya_New/SKAB - Skoltech Anomaly Benchmark/Normal Operation/file_48
Created dataset at: S

Processing /data/TimeSeriesResearch/datasets/Satya_New:  17%|████████████████████████████████▊                                                                                                                                                                    | 3/18 [00:01<00:07,  1.93it/s]

...

Created dataset at: Satya_New/turbofan/Unknown/file_2
Created dataset at: Satya_New/turbofan/Unknown/file_3
Created dataset at: Satya_New/NewYork/PM1/file_0
Created dataset at: Satya_New/NewYork/PM1/file_1
Created dataset at: Satya_New/NewYork/PM1/file_2
Created dataset at: Satya_New/NewYork/PM1/file_3
Created dataset at: Satya_New/NewYork/PM1/file_4
Created dataset at: Satya_New/IOCSV/Inner_and_Outer/file_0
Created dataset at: Satya_New/IOCSV/Inner_and_Outer/file_1
Created dataset at: Satya_New/IOCSV/Inner_and_Outer/file_2
Created dataset at: Satya_New/IOCSV/Inner_and_Outer/file_3


Processing /data/TimeSeriesResearch/datasets/Satya_New:  83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                | 15/18 [00:04<00:00,  3.02it/s]

Created dataset at: Satya_New/IOCSV/Inner_and_Outer/file_4
Created dataset at: Satya_New/Illinois/NO/file_0
Created dataset at: Satya_New/Illinois/NO/file_1
Created dataset at: Satya_New/Illinois/NO/file_2
Created dataset at: Satya_New/Illinois/NO/file_3
Created dataset at: Satya_New/Illinois/NO/file_4
Created dataset at: Satya_New/Illinois/NO/file_5
Created dataset at: Satya_New/Illinois/NO/file_6
Created dataset at: Satya_New/Illinois/NO/file_7
Created dataset at: Satya_New/Gyrometer/BuildingStairMove/file_0
Created dataset at: Satya_New/Gyrometer/BuildingStairMove/file_1
Created dataset at: Satya_New/Gyrometer/OutdoorMove/file_0
Created dataset at: Satya_New/Gyrometer/OutdoorMove/file_1
Created dataset at: Satya_New/Gyrometer/EscalatorMove/file_0
Created dataset at: Satya_New/Gyrometer/EscalatorMove/file_1
Created dataset at: Satya_New/Gyrometer/BuildingElevatorMove/file_0
Created dataset at: Satya_New/Gyrometer/BuildingElevatorMove/file_1
Created dataset at: Satya_New/Gyrometer/Flo

Processing /data/TimeSeriesResearch/datasets/Satya_New: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:05<00:00,  3.46it/s]

Created dataset at: Satya_New/IBCSV/Inner_and_Ball/file_2
Created dataset at: Satya_New/IBCSV/Inner_and_Ball/file_3


In [46]:
def load_h5py_dataset(h5_file=H5_FILE, window_size=None, stride=None, concat=False):
    datasets = defaultdict(list)
    def recursive_group_traversal(group, path=""):
        for key in group.keys():
            item = group[key]

            if isinstance(item, h5py.Group):
                recursive_group_traversal(item, path + "/" + key)
            elif isinstance(item, h5py.Dataset):
                label = path.split("/")[-1]
                dataset_name = "/".join(path.split("/")[1:-1])
                #print("label: ", label)

                datasets[dataset_name].append(
                    UnwindowedDataset(
                        data=item,
                        dataset_name=dataset_name,
                        descriptions=item.attrs.get("descriptions", []),
                        label=label
                    )
                )

    f = h5py.File(h5_file, 'r')
    recursive_group_traversal(f)

    if window_size and stride:
        for dataset_name in datasets:
            datasets[dataset_name] = [
                dataset.window(window_size=window_size, stride=stride)
                for dataset in datasets[dataset_name]
            ]

    if concat:
        for dataset_name in datasets:
            datasets[dataset_name] = torch.utils.data.ConcatDataset(
                datasets[dataset_name]
            )

    return datasets

In [47]:
# H5_FILE = '/data/TimeSeriesResearch/datasets/satya_new_data2.h5'
datasets = load_h5py_dataset(H5_FILE, window_size=128, stride=1, concat=True)

label:  BuildingElevatorMove
label:  BuildingElevatorMove
label:  BuildingStairMove
label:  BuildingStairMove
label:  EscalatorMove
label:  EscalatorMove
label:  FloorMove
label:  FloorMove
label:  FloorMove
label:  FloorMove
label:  FloorMove
label:  FloorMove
label:  FloorMove
label:  FloorMove
label:  IndoorMove
label:  OutdoorMove
label:  OutdoorMove
label:  Unknown
label:  Unknown
label:  Ball
label:  Ball
label:  Ball
label:  Ball
label:  Ball
label:  Ball
label:  Ball
label:  Ball
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
label:  CO
l

In [48]:
datasets.keys()

dict_keys(['Satya_New/Accelerometer', 'Satya_New/Airquality_pattern', 'Satya_New/BCSV', 'Satya_New/California', 'Satya_New/Delaware', 'Satya_New/Florida', 'Satya_New/Gyrometer', 'Satya_New/IBCSV', 'Satya_New/IOCSV', 'Satya_New/Illinois', 'Satya_New/Maryland', 'Satya_New/Michigan', 'Satya_New/NewYork', 'Satya_New/OCSV', 'Satya_New/SKAB - Skoltech Anomaly Benchmark', 'Satya_New/Texas', 'Satya_New/Wyoming', 'Satya_New/turbofan'])

In [56]:
datasets['Satya_New/SKAB - Skoltech Anomaly Benchmark'][10000]

Data(data=tensor([[2.0994e-01, 2.7113e-01, 2.4842e+00,  ..., 2.7816e+01, 2.3845e+02,
         1.2167e+02],
        [2.1342e-01, 2.7373e-01, 2.5717e+00,  ..., 2.7814e+01, 2.3814e+02,
         1.2234e+02],
        [2.1196e-01, 2.7043e-01, 1.3182e+00,  ..., 2.7814e+01, 2.1831e+02,
         1.2233e+02],
        ...,
        [2.1395e-01, 2.7222e-01, 2.2187e+00,  ..., 2.7901e+01, 2.1791e+02,
         1.2400e+02],
        [2.1506e-01, 2.7371e-01, 2.6990e+00,  ..., 2.7906e+01, 2.3070e+02,
         1.2400e+02],
        [2.1316e-01, 2.7334e-01, 2.6837e+00,  ..., 2.7906e+01, 2.3063e+02,
         1.2400e+02]], dtype=torch.float64), descriptions=['Shows a vibration acceleration from sensor 1 (Amount of g units)', 'Shows a vibration acceleration from sensor 2 (Amount of g units)', 'Shows the amperage on the electric motor (Ampere)', 'Represents the pressure in the loop after the water pump (Bar)', 'Shows the temperature of the engine body (The degree Celsius)', 'Represents the temperature of the flu

In [55]:
def get_unique_labels(dataset, max_samples=None):
    unique_labels = set()
    sample_count = len(dataset) if max_samples is None else min(max_samples, len(dataset))
    
    for i in range(sample_count):
        try:
            sample = dataset[i]
            # Extract label from the sample
            label = sample.label if hasattr(sample, 'label') else None
            unique_labels.add(label)
        except Exception as e:
            print(f"Error processing sample {i}: {e}")
    
    return unique_labels

# Use the function
dataset = datasets['Satya_New/Accelerometer']
print(f"Total samples in dataset: {len(dataset)}")

# Check first 1000 samples (adjust as needed)
unique_labels = get_unique_labels(dataset, max_samples=34058)
print(f"Unique labels found: {unique_labels}")

Total samples in dataset: 76625
Unique labels found: {'BuildingStairMove', 'FloorMove', 'BuildingElevatorMove', 'EscalatorMove'}


In [51]:
dataset[76621]

Data(data=tensor([[ 2.2140e-01, -1.2371e+00, -2.4791e-01],
        [ 2.4892e-01, -1.3045e+00, -5.1163e-02],
        [ 3.1166e-01, -1.3081e+00,  4.9255e-02],
        [ 4.2648e-01, -1.1856e+00,  2.3650e-03],
        [ 6.2762e-01, -1.0739e+00,  4.7607e-02],
        [ 8.2907e-01, -1.1360e+00,  1.9487e-01],
        [ 9.1180e-01, -1.3046e+00,  2.7827e-01],
        [ 9.7119e-01, -1.4410e+00,  2.8445e-01],
        [ 8.8055e-01, -1.4797e+00,  1.3676e-01],
        [ 7.3700e-01, -1.4396e+00,  6.6559e-02],
        [ 5.8009e-01, -1.2931e+00, -4.8584e-02],
        [ 4.3559e-01, -1.0605e+00, -1.2172e-01],
        [ 2.8610e-01, -7.7502e-01, -6.6803e-02],
        [ 1.5233e-01, -5.3307e-01,  7.3914e-02],
        [ 1.8661e-02, -3.5214e-01,  1.9035e-01],
        [-6.1646e-02, -2.7519e-01,  1.9536e-01],
        [-1.2268e-02, -3.2384e-01,  1.1102e-01],
        [ 1.1272e-01, -4.1740e-01,  2.5452e-02],
        [ 2.6991e-01, -6.1098e-01, -1.0990e-03],
        [ 3.8396e-01, -9.0237e-01, -1.6068e-02],
        [ 